In [2]:
BASE_DIR = '../'*3
RUN_DIR = BASE_DIR + 'code/CNN/cnn_ext_model/ext_model.ipynb'

In [3]:
%run {RUN_DIR}

In [5]:
class RnnBasicModel(CnnExtModel):
    pass

In [ ]:
def rnn_basic_alloc_rnn_layer(self, input_shape, hconfig):
    # 입력 출력의 시계열 인지 확인
    inseq = get_conf_param(hconfig, 'inseq', True)
    outseq = get_conf_param(hconfig, 'outseq', True)
    
    if inseq:
    # 시간대최대 길이timesteps1, 시간대별 입력 벡터 크기= timefeats
    # 시간축 크기를 나타내며 길이표시를 위해
    # 한칸더 할당되었음을 표시하기 위해 1을 붙임
    #timefeats -> x1~xn
    
        timesteps1, timefeats = input_shape
    else:
        timesteps1 = get_conf_param(hconfig, 'timesteps') + 1
        timefeats = np.prod(input_shape)
    # 순환백터의 크기     
    # hidden
    recur_size = get_conf_param(hconfig, 'recur_size')
    # 가중치와 편향파라미터 생성
    ex_inp_dim = timefeats + recur_size
    # 시간대별입력 + 순환벡터의 확장크기 => w_param
    # bias -> recur_size
#     weight, bias = self.alloc_param_pair([ex_inp_dim, recur_size])
    # timesteps 1 -> 나중에 forward or back 반복제어용 
    if outseq:
        # 시계열데이터일경우
        output_shape = [timesteps1, recur_size]
        
    else:
        # 시계열 데이터가 아닐경우 
        
        output_shape = [recur_size]
#     rnn_info = [inseq, outseq, timesteps1, timefeats, recur_size]
    params = nn.RNN(input_size=timefeats, hidden_size=recur_size,batch_first=True)
    rnn = RNN(params)
    return [rnn], output_shape
#     return {'w':weight, 'b': bias, 'info':rnn_info}, output_shape


RnnBasicModel.alloc_rnn_layer = rnn_basic_alloc_rnn_layer

In [4]:
# help(nn.RNN)

Help on class RNN in module torch.nn.modules.rnn:

class RNN(RNNBase)
 |  RNN(*args, **kwargs)
 |  
 |  Applies a multi-layer Elman RNN with :math:`\tanh` or :math:`\text{ReLU}` non-linearity to an
 |  input sequence.
 |  
 |  
 |  For each element in the input sequence, each layer computes the following
 |  function:
 |  
 |  .. math::
 |      h_t = \tanh(W_{ih} x_t + b_{ih} + W_{hh} h_{(t-1)} + b_{hh})
 |  
 |  where :math:`h_t` is the hidden state at time `t`, :math:`x_t` is
 |  the input at time `t`, and :math:`h_{(t-1)}` is the hidden state of the
 |  previous layer at time `t-1` or the initial hidden state at time `0`.
 |  If :attr:`nonlinearity` is ``'relu'``, then :math:`\text{ReLU}` is used instead of :math:`\tanh`.
 |  
 |  Args:
 |      input_size: The number of expected features in the input `x`
 |      hidden_size: The number of features in the hidden state `h`
 |      num_layers: Number of recurrent layers. E.g., setting ``num_layers=2``
 |          would mean stacking tw

In [1]:
import torch.nn as nn

In [ ]:
class RNN(nn.Module):
    def __init__(self,rnn):
        super().__init__()
        self.rnn = rnn
    def forward(self,x):    
        print(x.shape,'x.shapex.shapex.shape')
        output,hiddens=self.rnn(x)
        print(output.shape,'output.shapeoutput.shapeoutput.shape')
        print(hiddens.shape,'hiddens.shapehiddens.shapehiddens.shape')
#         print(out)
#         print(len(out))
#         output, hidden=self.rnn(x)
#         print(self.rnn(x))
        return output